# Майнинг Bitcoin: симуляция и сложность

В этом ноутбуке мы:
1. Реализуем двойной SHA-256 (SHA-256d) -- хеш-функцию Bitcoin
2. Построим и захешируем Genesis-блок
3. Промоделируем процесс майнинга
4. Исследуем зависимость числа попыток от сложности
5. Реализуем алгоритм корректировки сложности
6. Разберём nBits компактный формат

**Предварительные знания:** SHA-256 (CRYPTO-03/04), структура блока (BTC-03), Proof of Work (BTC-06)

## 1. Двойной SHA-256 (SHA-256d)

In [ ]:
import hashlib
import struct
import time
import random

def sha256d(data: bytes) -> bytes:
    """Двойной SHA-256 -- стандартная хеш-функция Bitcoin."""
    return hashlib.sha256(hashlib.sha256(data).digest()).digest()

# Проверка: хеш от пустых данных
empty_hash = sha256d(b"")
print(f"SHA-256d('') = {empty_hash.hex()}")
print(f"Длина: {len(empty_hash)} байт = {len(empty_hash) * 8} бит")

## 2. Genesis-блок Bitcoin

Построим заголовок Genesis-блока (Block #0) из известных значений и проверим, что хеш совпадает.

In [ ]:
# Genesis block header fields (80 bytes total)
version = struct.pack('<I', 1)  # Version 1, little-endian
prev_hash = bytes(32)  # 32 нулевых байта (нет предыдущего блока)
merkle_root = bytes.fromhex(
    '3ba3edfd7a7b12b27ac72c3e67768f617fc81bc3888a51323a9fb8aa4b1e5e4a'
)  # Little-endian merkle root единственной coinbase-транзакции
timestamp = struct.pack('<I', 1231006505)  # 2009-01-03 18:15:05 UTC
bits = struct.pack('<I', 0x1d00ffff)  # nBits -- начальная сложность
nonce = struct.pack('<I', 2083236893)  # Nonce, найденный Сатоси

# Собираем 80-байтный заголовок
genesis_header = version + prev_hash + merkle_root + timestamp + bits + nonce
print(f"Длина заголовка: {len(genesis_header)} байт")
print(f"Заголовок (hex): {genesis_header.hex()[:80]}...")

# Хешируем и проверяем
genesis_hash = sha256d(genesis_header)
# Bitcoin отображает хеши в big-endian (reversed)
genesis_hash_display = genesis_hash[::-1].hex()
print(f"\nGenesis block hash: {genesis_hash_display}")

expected = "000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f"
assert genesis_hash_display == expected, "Hash mismatch!"
print("Хеш совпадает с известным Genesis-блоком!")

## 3. Симуляция майнинга

Реализуем функцию `mine_block`, которая перебирает nonce до нахождения хеша меньше target.

In [ ]:
def mine_block(header_bytes: bytes, target: int, max_nonce: int = 2**20) -> tuple:
    """
    Майнинг блока: перебирает nonce от 0 до max_nonce.
    header_bytes -- 76 байт заголовка без nonce (последние 4 байта будут заменены).
    Возвращает (nonce, hash_hex) или (None, None) если не найден.
    """
    for nonce in range(max_nonce):
        # Подставляем nonce (little-endian, 4 байта)
        header = header_bytes[:76] + struct.pack('<I', nonce)
        hash_result = sha256d(header)
        hash_int = int.from_bytes(hash_result, 'big')
        if hash_int < target:
            return nonce, hash_result[::-1].hex()
    return None, None

# Тест: майним с лёгким target (много ведущих нулей не требуется)
easy_target = 2**248  # Нужен хеш с хотя бы одним ведущим нулевым байтом
header_no_nonce = genesis_header[:76]

start = time.time()
found_nonce, found_hash = mine_block(header_no_nonce, easy_target)
elapsed = time.time() - start

if found_nonce is not None:
    print(f"Блок найден!")
    print(f"Nonce: {found_nonce}")
    print(f"Hash:  {found_hash}")
    print(f"Время: {elapsed:.3f}с")
else:
    print("Блок не найден в пределах max_nonce")

## 4. Зависимость попыток от сложности

Промайним блоки с разной сложностью и построим график.

In [ ]:
# Майним с разными target-ами и записываем количество попыток
difficulties = []
nonce_counts = []
times = []

# Создаём случайный заголовок для чистоты эксперимента
random_header = bytes(range(76))  # 76 байт без nonce

print(f"{'Ведущие нули':<15} {'Target (hex)':<20} {'Nonce':<10} {'Время, с':<10}")
print("-" * 55)

for leading_zero_bits in [8, 12, 16, 20]:
    target = 2 ** (256 - leading_zero_bits)
    
    start = time.time()
    nonce, hash_hex = mine_block(random_header, target, max_nonce=2**22)
    elapsed = time.time() - start
    
    if nonce is not None:
        difficulties.append(leading_zero_bits)
        nonce_counts.append(nonce)
        times.append(elapsed)
        target_hex = f"2^{256 - leading_zero_bits}"
        print(f"{leading_zero_bits:<15} {target_hex:<20} {nonce:<10} {elapsed:<10.3f}")
    else:
        print(f"{leading_zero_bits:<15} -- не найден за 2^22 попыток")

print(f"\nКаждые +4 бита сложности увеличивают попытки в ~16 раз (2^4 = 16)")

In [ ]:
# Визуализация: количество попыток vs сложность
if difficulties:
    print("\nГрафик: Ведущие нулевые биты -> Количество попыток (nonce)")
    print("=" * 50)
    max_bar = 40
    max_nonce = max(nonce_counts) if nonce_counts else 1
    for d, n in zip(difficulties, nonce_counts):
        bar_len = int(n / max_nonce * max_bar)
        bar = '#' * bar_len
        print(f"{d:>3} бит | {bar:<{max_bar}} | {n:>8}")

## 5. Корректировка сложности

Реализуем алгоритм Bitcoin для пересчёта target каждые 2016 блоков.

In [ ]:
def calculate_new_target(old_target: int, actual_time_seconds: int) -> int:
    """
    Корректировка сложности Bitcoin каждые 2016 блоков.
    Expected time: 2016 * 10 * 60 = 1,209,600 секунд.
    Factor capped between 0.25x and 4x.
    """
    EXPECTED_TIME = 2016 * 10 * 60  # 1,209,600 секунд
    
    # Вычисляем фактор корректировки
    adjustment = actual_time_seconds / EXPECTED_TIME
    
    # Ограничиваем (предохранительные caps)
    adjustment = max(0.25, min(4.0, adjustment))
    
    new_target = int(old_target * adjustment)
    
    # Target не может превышать максимум
    MAX_TARGET = 0x00000000FFFF0000000000000000000000000000000000000000000000000000
    return min(new_target, MAX_TARGET)


# Пример: блоки шли быстрее (8 мин в среднем вместо 10)
EXPECTED = 2016 * 10 * 60
actual_fast = 2016 * 8 * 60  # 8 минут в среднем
actual_slow = 2016 * 12 * 60  # 12 минут в среднем

old_target = 0x00000000FFFF0000000000000000000000000000000000000000000000000000

new_target_fast = calculate_new_target(old_target, actual_fast)
new_target_slow = calculate_new_target(old_target, actual_slow)

print("Корректировка сложности")
print("=" * 60)
print(f"Old target:          {old_target:#066x}")
print()
print(f"Блоки по 8 мин (быстро):")
print(f"  Factor: {actual_fast/EXPECTED:.4f}")
print(f"  New target:        {new_target_fast:#066x}")
print(f"  Результат: target уменьшился -> СЛОЖНЕЕ")
print()
print(f"Блоки по 12 мин (медленно):")
print(f"  Factor: {actual_slow/EXPECTED:.4f}")
print(f"  New target:        {new_target_slow:#066x}")
print(f"  Результат: target увеличился -> ЛЕГЧЕ")

## 6. Симуляция 10 эпох с колебаниями хешрейта

In [ ]:
random.seed(42)  # Для воспроизводимости

# Начальные условия
target = 0x00000000FFFF0000000000000000000000000000000000000000000000000000
EXPECTED_TIME = 2016 * 10 * 60

print(f"{'Эпоха':<8} {'Avg мин/блок':<15} {'Фактор':<10} {'Направление':<15} {'Target (первые 8 hex)'}")
print("-" * 70)

for epoch in range(1, 11):
    # Симулируем случайное отклонение хешрейта (+/- 30%)
    hashrate_factor = random.uniform(0.7, 1.3)
    # Если хешрейт выше -- блоки быстрее
    avg_block_time = 10 / hashrate_factor  # минут
    actual_time = int(2016 * avg_block_time * 60)  # секунды
    
    # Корректируем
    new_target = calculate_new_target(target, actual_time)
    factor = actual_time / EXPECTED_TIME
    factor_capped = max(0.25, min(4.0, factor))
    
    direction = "сложнее" if new_target < target else "легче" if new_target > target else "без изменений"
    
    target_hex = f"{new_target:064x}"[:8]
    print(f"{epoch:<8} {avg_block_time:<15.2f} {factor_capped:<10.4f} {direction:<15} {target_hex}")
    
    target = new_target

print(f"\nСистема стремится поддерживать 10 мин/блок через автоматическую корректировку.")

## 7. nBits компактный формат: кодирование и декодирование

In [ ]:
def decode_nbits(nbits: int) -> int:
    """
    Декодирует compact nBits в полный 256-битный target.
    nBits = [1 байт экспонента][3 байта мантисса]
    target = мантисса * 2^(8 * (экспонента - 3))
    """
    exponent = (nbits >> 24) & 0xFF
    mantissa = nbits & 0x00FFFFFF
    
    # Обработка знакового бита мантиссы
    if mantissa & 0x800000:
        mantissa &= 0x7FFFFF
        # Отрицательный target (не используется в Bitcoin)
    
    if exponent <= 3:
        target = mantissa >> (8 * (3 - exponent))
    else:
        target = mantissa << (8 * (exponent - 3))
    
    return target


def encode_nbits(target: int) -> int:
    """
    Кодирует полный target обратно в compact nBits формат.
    """
    # Находим количество байт в target
    if target == 0:
        return 0
    
    target_bytes = target.to_bytes((target.bit_length() + 7) // 8, 'big')
    size = len(target_bytes)
    
    # Извлекаем 3 старших байта как мантиссу
    if size >= 3:
        mantissa = int.from_bytes(target_bytes[:3], 'big')
    else:
        mantissa = int.from_bytes(target_bytes.ljust(3, b'\x00'), 'big')
    
    # Если старший бит мантиссы установлен, сдвигаем
    if mantissa & 0x800000:
        mantissa >>= 8
        size += 1
    
    return (size << 24) | mantissa


# Тест: Genesis block nBits
genesis_nbits = 0x1d00ffff
decoded_target = decode_nbits(genesis_nbits)

print("nBits декодирование")
print("=" * 60)
print(f"nBits:      0x{genesis_nbits:08x}")
print(f"Экспонента: 0x{(genesis_nbits >> 24) & 0xFF:02x} = {(genesis_nbits >> 24) & 0xFF}")
print(f"Мантисса:   0x{genesis_nbits & 0xFFFFFF:06x}")
print(f"Target:     {decoded_target:#066x}")
print()

# Обратное кодирование
re_encoded = encode_nbits(decoded_target)
print(f"Re-encoded: 0x{re_encoded:08x}")
print(f"Match: {re_encoded == genesis_nbits}")

# Ещё пример: блок с высокой сложностью
print()
hard_nbits = 0x17034267
hard_target = decode_nbits(hard_nbits)
print(f"Пример высокой сложности:")
print(f"nBits:      0x{hard_nbits:08x}")
print(f"Экспонента: {(hard_nbits >> 24) & 0xFF}")
print(f"Target:     {hard_target:#066x}")
print(f"Difficulty:  {decoded_target / hard_target:.2f}")

## 8. Упражнения

### Упражнение 1: Майнинг с вариацией timestamp

Модифицируйте `mine_block`, чтобы при переполнении nonce (2^32) менялся timestamp на +1 секунду. Найдите блок с target = 2^(256-24).

In [ ]:
# Ваш код здесь
# Подсказка: после nonce=2^32, увеличьте timestamp в header_bytes
# и начните nonce с 0 снова


### Упражнение 2: Расчёт ожидаемого времени

Для хешрейта 100 TH/s и difficulty = 50,000,000,000,000 рассчитайте:
- Ожидаемое количество хешей до нахождения блока
- Ожидаемое время в часах
- Вероятность найти блок за 10 минут

In [ ]:
# Ваш код здесь
# Подсказка: E[hashes] = difficulty * 2^32
# E[time] = E[hashes] / hashrate
# P(block in 10 min) = 1 - e^(-lambda * t), lambda = hashrate / E[hashes]


### Упражнение 3: Декодирование реальных nBits

Декодируйте nBits из реальных блоков Bitcoin mainnet и вычислите difficulty для каждого.

In [ ]:
# Ваш код здесь
real_nbits_values = [
    0x1d00ffff,  # Genesis block (Block #0)
    0x1b04864c,  # Block #100,000 (2012)
    0x18009645,  # Block #400,000 (2016)
    0x17034267,  # Block #600,000 (2019)
]

genesis_target = decode_nbits(0x1d00ffff)
for nbits in real_nbits_values:
    target = decode_nbits(nbits)
    difficulty = genesis_target / target if target > 0 else float('inf')
    # Выведите: nBits, target (первые 16 hex), difficulty


## 9. Челлендж: Упрощённая симуляция Selfish Mining

Selfish mining -- стратегия, при которой майнер скрывает найденные блоки и публикует их в удобный момент, чтобы получить непропорционально большую долю наград.

Реализуйте упрощённую симуляцию:
- Честный майнер (hashrate = 70%) публикует блоки сразу
- Selfish майнер (hashrate = 30%) скрывает блоки
- Когда честный майнер находит блок, selfish майнер публикует свою скрытую цепочку (если она длиннее)
- Подсчитайте долю блоков selfish майнера за 10000 раундов

In [ ]:
# Ваш код здесь
# Подсказка: на каждом раунде случайно определяйте, кто нашёл блок
# (вероятность пропорциональна hashrate)
# Selfish майнер накапливает приватную цепочку
# и публикует её, когда она длиннее публичной
